In [1]:
!pip install gdown

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import sys
sys.path.append('..')
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
import os
import torchvision
from torchvision import datasets, models, transforms
from torch.optim.lr_scheduler import StepLR


from src.models import ModelResnet152dTimm
from src.dataloader import EuroSatDownloader
from src.training import train_fine_tuning, eval_method, eval_func
from src.dataloader import MiniImageNetDataSet, createEuroSatDataLoaders, EuroSatDownloader
from src.modelvis import visualize_models

plt.ion()

'''
The line cudnn.benchmark = True is typically used in deep learning projects that utilize the CUDA Deep Neural Network (cuDNN) library.
When cudnn.benchmark is set to True, it enables cuDNN to automatically find the best algorithm configuration for the specific 
input sizes and hardware being used. This can result in improved performance during training and inference.By enabling benchmarking, 
cuDNN will run a short benchmarking phase during the first iteration of the model to determine the optimal algorithm configuration. 
This configuration is then cached and used for subsequent iterations, leading to faster execution times.
It's important to note that enabling benchmarking may introduce some overhead during the initial benchmarking phase, so it is typically 
recommended to use it when the input sizes are consistent throughout the training process.
Overall, setting cudnn.benchmark to True can help optimize the performance of deep learning models that use cuDNN.
'''
torch.backends.cudnn.benchmark = True

'''
The line plt.ion() is a function call that activates interactive mode in matplotlib.
When interactive mode is enabled, any plot that is created will be displayed immediately 
and can be updated dynamically. This means that you can modify the plot after it is displayed, 
such as changing the data or adding annotations, and the changes will be reflected in real-time.
'''
plt.ion()

In [3]:
print(torchvision.__version__)
print(torch.__version__)

0.9.2
1.8.2


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


### Download EuroSat Database

In [5]:
extractDir = '..\\data'
url = "https://zenodo.org/records/7711810/files/EuroSAT_RGB.zip?download=1"
downloader = EuroSatDownloader(url, extractDir)
dataDir = downloader.download()

2024-01-07 01:05:02,863 - INFO - EuroSat database already exists.


# Hyperparameters

In [6]:
LEARNING_RATE = 0.0005
WEIGHT_DECAY = 0.00001
NUM_EPOCHS = 5
BATCH_SIZE = 2
MOMENTUM = 0.09
STEP_SIZE = 1
num_of_classes = 64
EPISODES = 5
GAMMA = 0.01

In [7]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
        transforms.RandomHorizontalFlip(),
        transforms.GaussianBlur(3, sigma=(0.1, 2.0)),
        transforms.ToTensor(),
        transforms.Normalize((0.3, 0.3, 0.2), (0.5, 0.6, 0.5)),
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
       transforms.Normalize((0.3, 0.3, 0.2), (0.5, 0.6, 0.5)),
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize((0.3, 0.3, 0.2), (0.5, 0.6, 0.5)),
    ]),
}

dataDir = '..\\data\\miniImageNet\\train'
modelPath = '..\\data\\models\\best_model_ModelResnet152dTimm_FixedTransforms.pth'

In [8]:
print('Please wait patiently, it may take some seconds...')
total_test_acc = 0.0

for episode in range(EPISODES):
    print(f"\nTraining Episode: {episode + 1}...")
    
    loaded_model = torch.load(modelPath, map_location=device)
    model = ModelResnet152dTimm(64)
    model.load_state_dict(loaded_model)
    model.model.fc = nn.Linear(model.model.fc.in_features, 5).to(device)

    for param in model.parameters():
        param.requires_grad = True
    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(params = model.parameters(), lr = LEARNING_RATE, momentum = MOMENTUM, weight_decay = WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = STEP_SIZE, gamma = GAMMA)
    dataloaders, class_names, dataset_sizes = createEuroSatDataLoaders(data_transforms, dataDir, split=0.25, batch_size = BATCH_SIZE)
    
    model, _, _ = train_fine_tuning(model, dataloaders, criterion, optimizer, scheduler, num_epochs = NUM_EPOCHS, learning_rate = LEARNING_RATE)
    test_loader = dataloaders['test']
    acc_test, test_loss = eval_method(net=model, data_loader=test_loader)
    total_test_acc += acc_test
    print(f"Episode {episode + 1}: Test Accuracy: {acc_test:.4f}")

average_test_acc = total_test_acc / EPISODES
print("\nAverage Test Accuracy:", average_test_acc)

Please wait patiently, it may take some seconds...

Training Episode: 1...


2024-01-07 01:05:07,003 - INFO - Loading pretrained weights from url (https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnet152d_ra2-5cac0439.pth)


epoch: 0, accuracy: 0.320000, avg. loss: 1.533763, test accuracy: 0.133333
epoch: 1, accuracy: 0.400000, avg. loss: 1.475761, test accuracy: 0.266667
epoch: 2, accuracy: 0.440000, avg. loss: 1.486629, test accuracy: 0.226667
Episode 1: Test Accuracy: 0.2267 num of images: 75

Training Episode: 2...


2024-01-07 01:05:25,206 - INFO - Loading pretrained weights from url (https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnet152d_ra2-5cac0439.pth)


epoch: 0, accuracy: 0.240000, avg. loss: 1.606491, test accuracy: 0.280000
epoch: 1, accuracy: 0.280000, avg. loss: 1.547183, test accuracy: 0.293333
epoch: 2, accuracy: 0.320000, avg. loss: 1.536868, test accuracy: 0.293333
Episode 2: Test Accuracy: 0.2933 num of images: 75

Training Episode: 3...


...

epoch: 0, accuracy: 0.240000, avg. loss: 1.630154, test accuracy: 0.226667
epoch: 1, accuracy: 0.160000, avg. loss: 1.622713, test accuracy: 0.266667
epoch: 2, accuracy: 0.280000, avg. loss: 1.595363, test accuracy: 0.266667
Episode 3: Test Accuracy: 0.2667 num of images: 75

Training Episode: 4...


2024-01-07 01:05:53,390 - INFO - Loading pretrained weights from url (https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnet152d_ra2-5cac0439.pth)


epoch: 0, accuracy: 0.240000, avg. loss: 1.619021, test accuracy: 0.226667
epoch: 1, accuracy: 0.160000, avg. loss: 1.590062, test accuracy: 0.306667
epoch: 2, accuracy: 0.240000, avg. loss: 1.588618, test accuracy: 0.280000
Episode 4: Test Accuracy: 0.2800 num of images: 75

Training Episode: 5...


2024-01-07 01:06:08,193 - INFO - Loading pretrained weights from url (https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnet152d_ra2-5cac0439.pth)


epoch: 0, accuracy: 0.240000, avg. loss: 1.601058, test accuracy: 0.213333
epoch: 1, accuracy: 0.160000, avg. loss: 1.583845, test accuracy: 0.213333
epoch: 2, accuracy: 0.400000, avg. loss: 1.565922, test accuracy: 0.240000
Episode 5: Test Accuracy: 0.2400 num of images: 75

Average Test Accuracy: 0.2613333333333333
